In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local[*]").getOrCreate()

In [3]:
from pyspark.sql import functions as f
from pyspark.sql import Window

In [4]:
data = sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")

In [5]:
data.take(10)

[Row(userId=13065, trackId=944906, artistId=978428, timestamp=1501588527),
 Row(userId=101897, trackId=799685, artistId=989262, timestamp=1501555608),
 Row(userId=215049, trackId=871513, artistId=988199, timestamp=1501604269),
 Row(userId=309769, trackId=857670, artistId=987809, timestamp=1501540265),
 Row(userId=397833, trackId=903510, artistId=994595, timestamp=1501597615),
 Row(userId=501769, trackId=818149, artistId=994975, timestamp=1501577955),
 Row(userId=601353, trackId=958990, artistId=973098, timestamp=1501602467),
 Row(userId=710921, trackId=916226, artistId=972031, timestamp=1501611582),
 Row(userId=6743, trackId=801006, artistId=994339, timestamp=1501584964),
 Row(userId=152407, trackId=913509, artistId=994334, timestamp=1501571055)]

In [6]:
meta.take(10)

[Row(type=u'track', Name=u'Smile', Artist=u'Artist: Josh Groban', Id=1223851),
 Row(type=u'track', Name=u'Chuni Ashkharhe Qez Nman 2015', Artist=u'Artist: Razmik Amyan', Id=1215486),
 Row(type=u'track', Name=u'Dark City', Artist=u'Artist: Machinae Supremacy', Id=1296462),
 Row(type=u'track', Name=u'Not Sensitive', Artist=u'Artist: Moby', Id=1249694),
 Row(type=u'artist', Name=u'Artist: Carlos Puebla', Artist=u'Artist: Carlos Puebla', Id=1352221),
 Row(type=u'track', Name=u'Thiz Gangsta Chit Iz Ourz', Artist=u'Artist: Tha Dogg Pound', Id=1217194),
 Row(type=u'track', Name=u'Ruffneck', Artist=u'Artist: Skrillex', Id=1245681),
 Row(type=u'track', Name=u'Incerc', Artist=u'Artist: Spike', Id=1193283),
 Row(type=u'artist', Name=u'Artist: Wallenbergs Whiskey Hell', Artist=u'Artist: Wallenbergs Whiskey Hell', Id=1333444),
 Row(type=u'track', Name=u'remix', Artist=u'Artist: Flo Rida', Id=1246378)]

In [7]:
findArtists = data.join(meta, data.artistId == meta.Id) \
    .where((f.col('userId') == 776748) & (f.col('type') == u'artist')) \
    .select('Artist', 'Name', 'type').distinct()

In [8]:
findArtists.take(10)

[Row(Artist=u'Artist: Blur', Name=u'Artist: Blur', type=u'artist'),
 Row(Artist=u'Artist: Linkin Park', Name=u'Artist: Linkin Park', type=u'artist'),
 Row(Artist=u'Artist: Clawfinger', Name=u'Artist: Clawfinger', type=u'artist'),
 Row(Artist=u'Artist: 3 Doors Down', Name=u'Artist: 3 Doors Down', type=u'artist'),
 Row(Artist=u'Artist: Serj Tankian', Name=u'Artist: Serj Tankian', type=u'artist'),
 Row(Artist=u'Artist: Three Days Grace', Name=u'Artist: Three Days Grace', type=u'artist'),
 Row(Artist=u'Artist: Gotthard', Name=u'Artist: Gotthard', type=u'artist'),
 Row(Artist=u'Artist: Iggy Pop', Name=u'Artist: Iggy Pop', type=u'artist'),
 Row(Artist=u'Artist: Nomy', Name=u'Artist: Nomy', type=u'artist'),
 Row(Artist=u'Artist: Nickelback', Name=u'Artist: Nickelback', type=u'artist')]

In [9]:
findTracks = data.join(meta, data.trackId == meta.Id) \
    .where((f.col('userId') == 776748) & (f.col('type') == u'track')) \
    .select('Artist', 'Name', 'type').distinct()

In [10]:
findTracks.take(10)

[Row(Artist=u'Artist: Thousand Foot Krutch', Name=u'Take It Out On Me', type=u'track'),
 Row(Artist=u'Artist: Linkin Park', Name=u'Numb', type=u'track'),
 Row(Artist=u'Artist: Papa Roach', Name=u'Getting Away With Murder', type=u'track'),
 Row(Artist=u'Artist: Rise Against', Name=u'Prayer Of The Refugee', type=u'track'),
 Row(Artist=u'Artist: Blur', Name=u'Girls and Boys', type=u'track'),
 Row(Artist=u'Artist: Korn', Name=u'Here To Stay', type=u'track'),
 Row(Artist=u'Artist: Three Days Grace', Name=u'I Hate Everything About You', type=u'track'),
 Row(Artist=u'Artist: 3 Doors Down', Name=u'Kryptonite', type=u'track'),
 Row(Artist=u'Artist: Green Day', Name=u'21 Guns', type=u'track'),
 Row(Artist=u'Artist: Green Day', Name=u'Kill The DJ', type=u'track')]

In [11]:
findAll = findArtists.union(findTracks) \
    .orderBy('Artist', 'type', 'Name') \
    .select('Artist', 'Name') \
    .take(40)

In [12]:
for val in findAll:
    print "%s %s" % val

Artist: 3 Doors Down Artist: 3 Doors Down
Artist: 3 Doors Down Kryptonite
Artist: 311 Artist: 311
Artist: 311 Beautiful disaster
Artist: Blur Artist: Blur
Artist: Blur Girls and Boys
Artist: Clawfinger Artist: Clawfinger
Artist: Clawfinger Nothing Going On
Artist: Disturbed Artist: Disturbed
Artist: Disturbed The Vengeful One
Artist: Gotthard Artist: Gotthard
Artist: Gotthard Eagle
Artist: Green Day Artist: Green Day
Artist: Green Day 21 Guns
Artist: Green Day Kill The DJ
Artist: Iggy Pop Artist: Iggy Pop
Artist: Iggy Pop Sunday
Artist: Korn Artist: Korn
Artist: Korn Here To Stay
Artist: Linkin Park Artist: Linkin Park
Artist: Linkin Park In The End
Artist: Linkin Park Numb
Artist: Lordi Artist: Lordi
Artist: Lordi Hard Rock Hallelujah
Artist: Nickelback Artist: Nickelback
Artist: Nickelback She Keeps Me Up
Artist: Nomy Artist: Nomy
Artist: Nomy Cocaine
Artist: Papa Roach Artist: Papa Roach
Artist: Papa Roach Getting Away With Murder
Artist: Rise Against Artist: Rise Against
Artist: Ri